<h1><center>Лабораторна робота 7.</center></h1>
<h2><center>Аналіз активності людини за відкритими даними мобільних телефонів</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-7"></a>

## Зміст

- [7.1. Завантаження та підготовка даних](#lab-7.1)
- [7.2. Застосування методу головних компонент](#lab-7.2)  
- [7.3. Застосування *k*-means](#lab-7.3)
- [7.4. Агломератична кластеризація](#lab-7.4)
- [7.5. Застосування класифікатора після кластеризації](#lab-7.5)
- [7.6. Класифікація із пониженням розмірності](#lab-7.6)

<a class="anchor" id="lab-7.1"></a>

## <span style="color:blue; font-size:1.2em;">7.1. Завантаження та підготовка даних</span>

[Повернутися до змісту](#lab-7)

У цій лабораторній робота Вам потрібно розібратися з тим, як працюють методи пониження розмірності й кластеризації даних. Крім того, ще раз попрактикуємося із задачею класифікації.

Тут ми працюватимемо з набором даних [Samsung Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Завантажте дані (чотири файли у форматі .txt) з [репозиторію](https://github.com/radiukpavlo/intelligent-data-analysis/tree/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR) на GitHub.

Дані взято з акселерометрів і гіроскопів мобільних телефонів Samsung Galaxy S3 (докладніше щодо ознак – за посиланням на UCI вище), також відомий вид активності людини з телефоном в кишені – чи ходила людина, стояла, лежала, сиділа або йшла вгору/вниз сходами.

Спершу представимо, що вид активності нам невідомий, і спробуємо кластеризувати людей лише на основі наявних свідчень. Далі розв'яжемо завдання визначення виду фізичної активності саме як задачу класифікації.

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 17

C:\Users\radiu\AppData\Local\Temp\ipykernel_33000\3543394545.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-darkgrid'])


In [10]:
# Завантаження даних, якщо код пишеться в редакторі з Anaconda
X_train = np.loadtxt("./samsung_HAR/samsung_train.txt")
y_train = np.loadtxt("./samsung_HAR/samsung_train_labels.txt").astype(int)

X_test = np.loadtxt("./samsung_HAR/samsung_test.txt")
y_test = np.loadtxt("./samsung_HAR/samsung_test_labels.txt").astype(int)

In [11]:
'''
import urllib3
from urllib.request import urlopen
# Альтернативне завантаження даних.
# Розкоментувати цей код, якщо завантаження тексктових даних вище не працює.

X_train_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_train.txt"
y_train_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_train_labels.txt"

X_test_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_test.txt"
y_test_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_test_labels.txt"

X_train_file = urllib3.request.urlopen(X_train_url)

X_train_pd = pd.read_csv("./samsung_HAR/samsung_train.txt")
data = urlopen(X_train_url)

np.loadtxt(X_train_url)
y_train = np.loadtxt(y_train_url)
X_test = np.loadtxt(X_test_url)
y_test = np.loadtxt(y_test_url)
'''

'\nimport urllib3\nfrom urllib.request import urlopen\n# Альтернативне завантаження даних.\n# Розкоментувати цей код, якщо завантаження тексктових даних вище не працює.\n\nX_train_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_train.txt"\ny_train_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_train_labels.txt"\n\nX_test_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_test.txt"\ny_test_url = "https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR/samsung_test_labels.txt"\n\nX_train_file = urllib3.request.urlopen(X_train_url)\n\nX_train_pd = pd.read_csv("./samsung_HAR/samsung_train.txt")\ndata = urlopen(X_train_url)\n\nnp.loadtxt(X_train_url)

In [12]:
# Перевіримо розмірності
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризації нам не потрібен вектор відповідей, тому будемо працювати з об'єднанням навчальної та тестової вибірок. Об'єднаємо `X_train` з `X_test` та `y_train` з `y_test`.

In [13]:
X = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])

Визначимо кількість унікальних значень міток цільового класу.

In [14]:
np.unique(y)

array([1, 2, 3, 4, 5, 6])

In [15]:
n_classes = np.unique(y).size
n_classes

6

Ці мітки [відповідають](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names) таким значенням:
- 1 – ходьба;
- 2 – підйом вгору сходами;
- 3 – спуску вниз сходами;
- 4 – сидіння;
- 5 – стояння;
- 6 – лежання.

Виконаємо масштабування вибірки за допомогою `StandardScaler` з параметрами за замовчуванням:

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

------------------------

<a class="anchor" id="lab-7.2"></a>

## <span style="color:blue; font-size:1.2em;">7.2. Застосування методу головних компонент</span>

[Повернутися до змісту](#lab-7)

### <span style="color:red; font-size:1.5em;">Завдання 1</span>


**Для всіх варіантів:**

**Виконайте пониження розмірності даних за допомогою PCA. Залишіть таку кількість компонент, щоби пояснити як мінімум 90% дисперсії оброблених (масштабованих) даних. Використовуйте масштабовану вибірку й зафіксуйте random_state (константа RANDOM_STATE).**

**Для варіантів 1–2:**

Застосуйте інкрементний PCA (`sklearn.decomposition.IncrementalPCA`) до вхідного набору даних, щоб ефективно керувати використанням пам’яті. Порівняйте результати зі стандартним PCA з погляду поясненої дисперсії та ефективності кластеризації за допомогою k-середніх. Обговоріть компроміси між інкрементним та звичайним PCA.

**Для варіантів 3–4:**

Використовуйте ядерний PCA (`sklearn.decomposition.KernelPCA`) з різними ядрами, такими як радіально-базисна функція (RBF), поліноміальна та сигмоїдна. Проаналізуйте, як вибір ядра впливає на результати кластеризації, і візуалізуйте дані в зменшеному просторі. Наведіть приклади, за яких Kernel PCA може бути більш вигідним, ніж стандартний PCA.

**Для варіантів 5–6:**

Застосуйте розріджений PCA (`sklearn.decomposition.SparsePCA`), щоб зменшити розрідженість компонентів, що полегшить їх інтерпретацію. Порівняйте його ефективність зі звичайним PCA з погляду ефективності кластеризації та інтерпретації компонентів. Обговоріть важливість розрідженості компонентів PCA в контексті розпізнавання людської діяльності.

**Для варіантів 7–8:**

Проведіть експерименти з різною кількістю компонентів в PCA (`sklearn.decomposition.PCA`) і проаналізуйте, як це впливає на ефективність кластеризації, використовуючи такі метрики, як оцінка силуету. Обговоріть концепцію «методу ліктя» при визначенні оптимальної кількості компонент.

**Для варіантів 9–10:**

Порівняйте PCA з лінійним дискримінантним аналізом (`sklearn.discriminant_analysis.LinearDiscriminantAnalysis`) для зменшення розмірності. Оцініть та порівняйте їх ефективність у задачах кластеризації та класифікації за відповідними метриками, обговоривши відмінності між підходами PCA (некерованим) та LDA (керованим).

**Для варіантів 11–12:**

Використовуйте PCA, щоб звести набір даних до трьох вимірів і візуалізувати кластери за допомогою діаграми розсіювання (scatter plot). Продемонструйте як різні види діяльності розділені в зменшеному просторі та визначте чи допомагає PCA виявити чіткі закономірності або відмінності між різними видами діяльності.

**Для варіантів 13–14:**

Застосуйте PCA з обертанням Varimax для отримання більш інтерпретованої структури з більш чіткими навантаженнями на ознаки. Обговоріть, як ротація впливає на інтерпретацію головних компонент та її корисність у розвідувальному аналізі.

**Для варіантів 15–16:**

Поєднайте PCA з методами відбору ознак, такими як SelectKBest та SelectPercentile (`sklearn.feature_selection`). Проведіть експерименти щодо того, як така комбінація впливає на продуктивність алгоритмів кластеризації та класифікації, й обговоріть баланс між відбором ознак і зменшенням розмірності.

**Для варіантів 17–18:**

Застосуйте PCA окремо до різних підмножин даних (наприклад, на основі різних видів діяльності) і порівняйте результати. Проаналізуйте як працює PCA на однорідних і неоднорідних підмножинах даних і які наслідки це має для розпізнавання активності.

**Для варіантів 19–20:**

Яку мінімальну кількість головних компонент потрібно виділити, щоб пояснити 90% дисперсії початкових (масштабованих) даних? Виведіть номер головної компоненти, на яку припадає найбільший % дисперсії (для 90% дисперсії). Скільки відсотків дисперсії припадає на першу головну компоненту?

------------------------

<a class="anchor" id="lab-7.3"></a>

## <span style="color:blue; font-size:1.2em;">7.3. Застосування *k*-means</span>

[Повернутися до змісту](#lab-7)

Виконайте кластеризацію даних методом `Kmeans`, навчивши модель за даними зі зниженою за рахунок PCA розмірністю. Тут потрібно шукати саме 6 кластерів, але загалом у задачах без вчителя ми не знаємо, яку кількість кластерів треба шукати.

Параметри:

- **n_clusters** = n_classes (кількість унікальних міток цільового класу);
- **n_init** = 100;
- **random_state** = RANDOM_STATE (для відтворюваності результату).

Інші параметри залишаються за замовчуванням.

### <span style="color:red; font-size:1.5em;">Завдання 2</span>


**Для варіантів 1–2:**

Побудуйте нові моделі на основі інших методів ініціалізації k-середніх (наприклад, k-means++, random) у `sklearn.cluster.KMeans`. Визначте, як ініціалізація впливає на кінцевий результат за метриками кластеризації та швидкість збіжності (convergence).

**Для варіантів 3–4:**

Змінюйте кількість кластерів у k-середніх та проведіть оцінювання результати кластеризації за допомогою таких метрик, як Silhouette Score та Davies-Bouldin index. Визначте оптимальну кількість кластерів та оцініть вплив цього значення на результат кластеризації.

**Для варіантів 5–6:**

Порівняйте продуктивність і обчислювальну ефективність k-середніх з Mini Batch K-means (`sklearn.cluster.MiniBatchKMeans`). Проведіть експерименти зі сценаріями, за яких Mini Batch K-means є кращим за стандартні k-means, особливо з точки зору масштабованості та часу обчислень.

**Для варіантів 7–8:**

Виконайте кластеризацію k-середніх за даними, розмірність яких була понижена за допомогою tSNE. Порівняйте результати кластеризації з k-середніми, що виконана за повним набором даних. Поясніть, як зменшення розмірності впливає на результат кластеризації.

**Для варіантів 9–10:**

Проведіть експерименти з різними метриками відстані в k-середніх (euclidean, squared euclidean, manhattan, chebyshev, canberra, chi-square) та проаналізуйте їхній вплив на результати кластеризації. Поясніть придатність кожної метрики для даного набору даних і характеру даних про людську діяльність.

**Для варіантів 11–12:**

Використайте k-середні для виявлення аномалій у наборі даних. Визначте, як кластеризація може бути використана для виявлення аномалій, а також запропонуйте шляхи вирішення проблем, що пов'язані з використанням k-середніх для цього завдання.

**Для варіантів 13–14:**

Реалізувати версію k-середніх, яка дає змогу зважувати ознаки (feature weighting), підкреслюючи певні ознаки більше, ніж інші. Визначте, як зважування ознак впливає на результати кластеризації та його потенційні переваги для розпізнавання активності.

**Для варіантів 15–16:**

Проведіть експерименти з різними критеріями збіжності k-середніх (change in inertia, maximum number of iterations) і проаналізуйте їхній вплив на кінцевий результат кластеризації та продуктивність алгоритму. Обґрунтуйте важливість встановлення відповідних критеріїв збіжності в k-середніх.

**Для варіантів 17–18:**

Використайте різні метрики (Adjusted Rand Index, Adjusted Mutual Information, Silhouette) для оцінювання якості кластеризації за методом k-середніх. Порівняйте результати кластеризації з істинними мітками (true labels), щоб оцінити продуктивність і проаналізуйте обмеження використання зовнішньої валідації в неконтрольованому навчанні.

**Для варіантів 19–20:**

Застосуйте метод k-середніх до ознак, що перетворені за допомогою методів Box-Cox та Yeo-Johnson (`sklearn.preprocessing`). Визначте, як різні перетворення впливають на результати кластеризації та важливість вибору відповідних перетворень для набору даних.


------------------------

<a class="anchor" id="lab-7.4"></a>

## <span style="color:blue; font-size:1.2em;">7.4. Агломератична кластеризація</span>

[Повернутися до змісту](#lab-7)

Спробуємо ще один алгоритм, що входить до групи ієрархічної кластеризації – агломеративну кластеризацію [`AgglomerativeClustering()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html).

### <span style="color:red; font-size:1.5em;">Завдання 3</span>


**Для варіантів 1–2:**

Проведіть експерименти з різними критеріями зв’язку в Агломеративній кластеризації (`sklearn.cluster.AgglomerativeClustering`), такими як Ward, complete, average, та single linkage. Визначте, як кожен критерій впливає на структуру кластеризації, особливо в контексті набору даних про людську діяльність.

**Для варіантів 3–4:**

Виконайте Агломеративну кластеризацію за даними, що попередньо оброблені за допомогою PCA та t-SNE. Визначте, як методи зменшення розмірності впливають на результати кластеризації та їхню корисність для виявлення прихованих закономірностей у даних.

**Для варіантів 5–6:**

Реалізуйте схему перехресної перевірки (cross-validation) для оцінки стабільності результатів Агломеративної кластеризації. Проаналізуйте методологію застосування перехресної перевірки в контексті навчання без учителя.

**Для варіантів 7–8:**

Включіть в Агломеративну кластеризацію обмеження зв’язності на основі k-найближчих сусідів. Визначте, як введення цих обмежень впливає на форму й розмір кластерів та загальну ефективність кластеризації.

**Для варіантів 9–10:**

Реалізуйте дендрограми (dendrograms) для результатів Агломеративної кластеризації та використайте їх для дослідження ієрархічної структури кластерів. Визначте, як аналіз дендрограми може допомогти у виборі оптимальної кількості кластерів і яку інформацію він надає про дані.

**Для варіантів 11–12:**

Використовуйте кластери, сформовані Агломеративною кластеризацією, як ознаки в моделі навчання з учителем (supervised learning model). Визначте, як цей підхід може підвищити продуктивність моделі та надати уявлення про взаємозв’язки між ознаками.

**Для варіантів 13–14:**

Обрахуйте Adjusted Rand Index для розбиття на два кластери за AgglomerativeClustering та MiniBatchKmeans з параметрами із пункту 7.3. Порівняйте значення ARI, знайденого для AgglomerativeClustering та MiniBatchKmeans.

**Для варіантів 15–16:**

Застосуйте різні методи нормалізації (StandardScaler, MinMaxScaler та RobustScaler) до набору даних перед виконанням Агломеративної кластеризації. Оцініть за метриками кластеризації (див. лекційну записку 7), як нормалізація впливає на результати кластеризації, й проаналізуйте важливість нормалізації ознак в ієрархічній кластеризації.

**Для варіантів 17–18:**

Використайте індекси валідності кластерів, такі як silhouette score, Calinski-Harabasz index, and Davies-Bouldin index, для оцінки якості кластерів, сформованих Агломеративною кластеризацією. Визначте переваги та обмеження цих індексів в оцінці якості кластеризації.

**Для варіантів 19–20:**

Проведіть експерименти з різними метриками відстані (euclidean, squared euclidean, manhattan, chebyshev, canberra, chi-square) в Агломеративній кластеризації. Проаналізуйте вплив цих метрик на результат кластеризації, особливо для розуміння людської діяльності.


------------------------

<a class="anchor" id="lab-7.5"></a>

## <span style="color:blue; font-size:1.2em;">7.5. Застосування класифікатора після кластеризації</span>

[Повернутися до змісту](#lab-7)

Варто відзначити, що попереднє завдання не надто добре розв'язується саме як задача кластеризації, якщо виділяти лише кілька кластерів (>2). Спробуємо тепер розв'яжемо задачу класифікації; пам'ятаємо, що дані у нас розмічені.

### <span style="color:red; font-size:1.5em;">Завдання 4</span>


**Для варіантів 1–2:**

Застосуйте різні класифікатори (SVM, Random Forest, Logistic Regression) до кластеризованих даних та порівняйте їхню ефективність. Проаналізуйте, як кластеризація як етап попередньої обробки впливає на точність класифікації та складність моделі.

**Для варіантів 3–4:**

Використайте деревовидні класифікатори, як от випадковий ліс та беггінг дерев ріщень після кластеризації для оцінювання важливості ознак. Проаналізуйте, як кластеризація впливає на важливість різних ознак у задачі класифікації.

**Для варіантів 5–6:**

Налаштуйте для `sklearn.svm.LinearSVC` гіперпараметри за допомогою `GridSearchCV` (cv=3). Виконайте навчання нового `StandardScaler` за навчальною вибіркою (з усіма початковими ознаками), застосуйте масштабування до тестової вибірки. Який вид активності LinearSVC визначає найгірше за метрикою $F_1$-score? Виведіть назву відповідної ознаки (вид активності) та значення відповідної метрики за допомогою коду.

**Для варіантів 7–8:**

Використайте `sklearn.model_selection.GridSearchCV` для налаштування гіперпараметрів класифікатора `sklearn.svm.LinearSVC`, застосованого після кластеризації. Проаналізуйте вплив кластеризації на оптимальні налаштування гіперпараметрів і продуктивність моделі.

**Для варіантів 9–10:**

Розгляньте мітки кластерів як нові ознаки та додайте їх до початкового набору ознак. Обрахуйте частку правильних відповідей (accuracy) класифікатора `sklearn.svm.LinearSVC`? Виведіть значення відповідної метрики за допомогою коду. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як така форма інженерії ознак впливає на продуктивність різних класифікаторів.

**Для варіантів 11–12:**

Проведіть експерименти з ансамблевими моделями та дослідіть різні метрики оцінювання (accuracy, precision, recall, $F_1$-score, ROC-AUC) для класифікаторів, застосованих після кластеризації. Проаналізуйте, як кластеризація впливає на ці метрики та наслідки для оцінки моделі.

**Для варіантів 13–14:**

Порівняйте продуктивність класифікатора `sklearn.svm.LinearSVC`, застосованого до даних, що кластеризовані за допомогою різних алгоритмів (k-середніх, агломеративної кластеризації, DBSCAN). Виведіть та проаналізуйте результати класифікації за допомогою classification_report.

**Для варіантів 15–16:**

Застосуйте методи зменшення розмірності (PCA, LDA та t-SNE) перед класифікацією та оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як така попередня обробка впливає на продуктивність класифікатора. Обговоріть переваги та недоліки зменшення розмірності в контексті кластеризованих даних.

**Для варіантів 17–18:**

Реалізуйте ансамбль стекування (`sklearn.ensemble.StackingClassifier`) з використанням різних базових класифікаторів за кластеризованими даними. Проаналізуйте продуктивність стекової моделі та обговоріть синергетичні ефекти поєднання декількох класифікаторів у цьому контексті.

**Для варіантів 19–20:**

Застосуйте ансамблеві методи (Gradient Boosting та AdaBoost з `sklearn.ensemble`) за кластеризованими даними. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як ансамблеві методи працюють у порівнянні з простими моделями (k-nearist neighbors та logistic regression) та роль кластеризації в покращенні роботи ансамблю.

------------------------

<a class="anchor" id="lab-7.6"></a>

## <span style="color:blue; font-size:1.2em;">7.6. Класифікація із пониженням розмірності</span>

[Повернутися до змісту](#lab-7)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>


**Для варіантів 1–2:**

Поєднайте метод PCA з методом вибору ознак, наприклад, SelectFromModel (`sklearn.feature_selection.SelectFromModel`), використовуючи SVM-класифікатор. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як ця комбінація впливає на ефективність класифікації та визначте баланс між зменшенням розмірності та важливістю ознак.

**Для варіантів 3–4:**

Застосуйте попередню обробку методом PCA перед використанням ансамблевих класифікаторів RandomForest та GradientBoosting (`sklearn.ensemble`). Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC) вплив PCA на продуктивність ансамблевих класифікаторів та проаналізуйте, як PCA може потенційно покращити ансамблеве навчання.

**Для варіантів 5–6:**

Використайте ядерний PCA (decomposition.KernelPCA) як частину конвеєра (`sklearn.pipeline.Pipeline`) і виконайте налаштування гіперпараметрів за допомогою GridSearchCV (`sklearn.model_selection.GridSearchCV`). Проаналізуйте, як зменшений простір ознак впливає на оптимальні значення гіперпараметрів і продуктивність класифікатора.

**Для варіантів 7–8:**

Проведіть експерименти з PCA, зберігаючи різні рівні дисперсії (90%, 95% і 99%), і проаналізуйте, як цей вибір впливає на продуктивність класифікатора. Зробіть висновки щодо компромісів між зменшенням розмірності та збереженням інформації.

**Для варіантів 9–10:**

Застосуйте нелінійні класифікатори k-nearist neighbors та SVM після застосування PCA. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як PCA впливає на продуктивність нелінійних моделей та проаналізуйте взаємодію між лінійним зменшенням розмірності та нелінійною класифікацією.

**Для варіантів 11–12:**

Поєднайте з методами вирівнювання дисбалансу класів [`imblearn.over_sampling.SMOTE`](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html) та class weight adjustment. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як ці методи впливають на продуктивність класифікатора на даних, перетворених методом PCA, і проаналізуйте проблеми обробки дисбалансу класів у просторах зі зменшеною розмірністю.

**Для варіантів 13–14:**

Використайте метод PCA, а потім класифікатор k-nearist neighbors з різними метриками відстані (euclidean, squared euclidean, manhattan, chebyshev, minkowski, and chi-square). Дослідіть, як PCA впливає на ефективність різних метрик відстані в класифікаторі KNN.

**Для варіантів 15–16:**

Застосуйте різні методи масштабування (StandardScaler, MinMaxScaler, and RobustScaler) перед PCA та оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC) їхній вплив на роботу класифікатора випадкового лісу.

**Для варіантів 17–18:**

Застосуйте `decomposition.TruncatedSVD` до початкових даних перед застосуванням беггінгу логістичної регресії з різним типом регуляризації (L1 та L2). Проаналізуйте, як регуляризація взаємодіє зі зменшеним простором ознак, створеним за допомогою PCA.

**Для варіантів 19–20:**

Використайте ансамблевий класифікатор беггінгу логістичної регресії зі стекуванням (`sklearn.ensemble.StackingClassifier`), де кожна базова модель навчається на різній кількості компонентів PCA. Оцініть за метриками класифікації (accuracy, precision, recall, $F_1$-score, ROC-AUC), як об'єднання моделей, навчених на різних рівнях зменшення розмірності, впливає на загальну продуктивність.
